In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline, TrainingArguments, Trainer,DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.utils.class_weight import compute_class_weight
import wandb
import torch


In [2]:
!pip install transformers

In [3]:
!pip install evaluate
import evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [4]:
!pip install datasets


In [6]:
df = pd.read_json("/content/jutsus.jsonl",lines=True)

In [7]:
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...
1,Anātman,Ninjutsu,Gaara sends a wave of sand towards his opponen...
2,Amputation Punishment,"Bukijutsu, Taijutsu",Hidan attacks the enemy many times with his sc...
3,Animal Path,"Kekkei Genkai, Ninjutsu, Dōjutsu",This article is about the ability. For other ...
4,Antlion Ninja Arts: Ephemeral,"Ninjutsu, Kinjutsu",This article is about the anime-only kinjutsu ...


In [8]:
# A function to eliminate any other type except those 3
def simplify_jutsu(jutsu):
  if "Genjutsu" in jutsu:
    return "Genjutsu"
  elif "Taijutsu" in jutsu:
    return "Taijutsu"
  elif "Ninjutsu" in jutsu:
    return "Ninjutsu"


In [9]:
df['jutsu_simple']= df['jutsu_type'].apply(simplify_jutsu)

In [10]:
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_simple
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...,Taijutsu
1,Anātman,Ninjutsu,Gaara sends a wave of sand towards his opponen...,Ninjutsu
2,Amputation Punishment,"Bukijutsu, Taijutsu",Hidan attacks the enemy many times with his sc...,Taijutsu
3,Animal Path,"Kekkei Genkai, Ninjutsu, Dōjutsu",This article is about the ability. For other ...,Ninjutsu
4,Antlion Ninja Arts: Ephemeral,"Ninjutsu, Kinjutsu",This article is about the anime-only kinjutsu ...,Ninjutsu


In [11]:
df['text'] = df['jutsu_name'] + " " + df['jutsu_type']
df['jutsus_type'] = df["jutsu_simple"]
df=df[['text','jutsus_type']]
df = df.dropna()

In [12]:
df.head()

,text,jutsus_type
0,10 Hit Combo Taijutsu,Taijutsu
1,Anātman Ninjutsu,Ninjutsu
2,"Amputation Punishment Bukijutsu, Taijutsu",Taijutsu
3,"Animal Path Kekkei Genkai, Ninjutsu, Dōjutsu",Ninjutsu
4,"Antlion Ninja Arts: Ephemeral Ninjutsu, Kinjutsu",Ninjutsu


In [13]:
text_column_name = 'text'
label_column_name = 'jutsus_type'

In [14]:
# Label Encoding

le = preprocessing.LabelEncoder()
le.fit_transform(df[label_column_name].tolist())

array([2, 1, 2, ..., 2, 2, 2])

In [15]:
#Looping on every class in the labels and returning a dictionary index : class name
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}
label_dict

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [16]:
df['label'] =  le.transform(df[label_column_name].tolist())

In [17]:
df.head()

,text,jutsus_type,label
0,10 Hit Combo Taijutsu,Taijutsu,2
1,Anātman Ninjutsu,Ninjutsu,1
2,"Amputation Punishment Bukijutsu, Taijutsu",Taijutsu,2
3,"Animal Path Kekkei Genkai, Ninjutsu, Dōjutsu",Ninjutsu,1
4,"Antlion Ninja Arts: Ephemeral Ninjutsu, Kinjutsu",Ninjutsu,1


In [18]:
test_size = 0.2
# The usage of stratify here is to make sure the model split each class
df_train,df_test=train_test_split(df,test_size=test_size,stratify=df['label'])


In [19]:
model_name = "distilbert/distilbert-base-uncased"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples[text_column_name],padding="max_length" ,truncation=True)

In [22]:
# Converting the pd df to a huggingface dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

tokenized_train = train_dataset.map(lambda examples : preprocess_function(tokenizer,examples),batched=True)
tokenized_test = test_dataset.map(lambda examples : preprocess_function(tokenizer,examples),batched=True)

Map:   0%|          | 0/2214 [00:00<?, ? examples/s]

Map:   0%|          | 0/554 [00:00<?, ? examples/s]

In [23]:
num_labels = len(le.classes_)  # Number of unique labels in your dataset
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
training_args = TrainingArguments(
    output_dir='./results',              # Directory to save model outputs
    num_train_epochs=3,                  # Number of training epochs
    per_device_train_batch_size=16,      # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    warmup_steps=500,                    # Warmup steps for learning rate
    weight_decay=0.01,                   # Weight decay for regularization
    logging_dir='./logs',                # Directory for logs
    logging_steps=10,                    # Log every 10 steps
)

In [25]:
metric = evaluate.load('accuracy')

In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rahman-ihab1 (rahman-ihab1-cairo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.250100
20,1.224300
30,1.174200
40,1.095100
50,0.999700
60,0.872500
70,0.784200
80,0.747600
90,0.582300
100,0.537200


TrainOutput(global_step=417, training_loss=0.2528947669860842, metrics={'train_runtime': 403.1899, 'train_samples_per_second': 16.474, 'train_steps_per_second': 1.034, 'total_flos': 879864152721408.0, 'train_loss': 0.2528947669860842, 'epoch': 3.0})

In [29]:
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.0006761497934348881, 'eval_accuracy': 1.0, 'eval_runtime': 8.4214, 'eval_samples_per_second': 65.785, 'eval_steps_per_second': 4.156, 'epoch': 3.0}


In [30]:
# Function to predict the label for a given text
def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    device = next(model.parameters()).device  # Get the model's device
    inputs = {k: v.to(device) for k, v in inputs.items()} #Move all input tensors to the device
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted label index
    predicted_label_idx = torch.argmax(logits, dim=1).item()

    # Map the index to the original label
    predicted_label = le.inverse_transform([predicted_label_idx])[0]

    return predicted_label

In [33]:
new_text = df_test['text'].iloc[0]
predicted_label = predict(new_text)
print(f"The predicted label is: {predicted_label}")

The predicted label is: Ninjutsu
